In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [ ]:
research_dataset = pd.read_csv('/content/drive/MyDrive/학부연구생/패션/output_dataset/research_preprocessing_out.csv')
research_dataset.head()

,E_id,era,style,gender,season,situation,fit,dark_bright,cold_warm,heavy_light,...,r_gender,age,mar,job,fancy_normal,sexual_unisexual,income,tranditional_trandy,formal_casual,active_gentle
0,1,1950,ivy,M,spring_fall,work,tight_fit,bright,warm,light,...,man,50_59,not_married,office,fancy,unisexual,600_,trandy,casual,active
1,2,1950,ivy,M,spring_fall,daily,tight_fit,bright,warm,light,...,man,50_59,not_married,office,fancy,unisexual,600_,trandy,casual,active
2,3,1950,ivy,M,summer,date,suitable_fit,dark,warm,light,...,man,50_59,not_married,office,fancy,unisexual,600_,trandy,casual,active
3,4,1960,mods,M,summer,work,tight_fit,bright,warm,light,...,man,50_59,not_married,office,fancy,unisexual,600_,trandy,casual,active
4,5,1960,mods,M,spring_fall,date,tight_fit,dark,cold,heavy,...,man,50_59,not_married,office,fancy,unisexual,600_,trandy,casual,active


In [ ]:
df = research_dataset.copy()
df = df.set_index('E_id')

In [ ]:
# feature와 label 나누기
X = df.drop(['Q5'], axis=1)
y = df['Q5']

In [ ]:
# one hot encoding
X = pd.get_dummies(X)
X.head()

,era,style_athleisure,style_bodyconscious,style_bold,style_cityglam,style_classic,style_disco,style_ecology,style_feminine,style_genderless,...,income_400_500,income_500_600,income_600_,income__200,tranditional_trandy_tranditional,tranditional_trandy_trandy,formal_casual_casual,formal_casual_formal,active_gentle_active,active_gentle_gentle
E_id,,,,,,,,,,,,,,,,,,,,,
1,1950,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0
2,1950,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0
3,1950,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0
4,1960,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0
5,1960,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0


In [ ]:
# split X and y into traning and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
#모델 종류
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#default 학습 결과

before_result = {'logistic':0, 'abc': 0, 'gb': 0}

logistic_model = LogisticRegression()
logistic_model.fit(X_train,y_train)
logistic_y_pred = logistic_model.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, logistic_y_pred))
before_result['logistic']=metrics.accuracy_score(y_test, logistic_y_pred)

abc_model = AdaBoostClassifier()
abc_model.fit(X_train,y_train)
abc_y_pred = abc_model.predict(X_test)
# Model Accuracy
print("AdaBoost Classifier Accuracy:",metrics.accuracy_score(y_test, abc_y_pred))
before_result['abc']=metrics.accuracy_score(y_test, abc_y_pred)

gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train, sample_weight=None, monitor=None)
gb_y_pred = gb_model.predict(X_test)
#Model Accuracy
print("GradientBoosting Classifier Accuracy:",metrics.accuracy_score(y_test, gb_y_pred))
before_result['gb']=metrics.accuracy_score(y_test, gb_y_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.723520961211963
AdaBoost Classifier Accuracy: 0.7284837403682904
GradientBoosting Classifier Accuracy: 0.7269165469505028


In [ ]:
#logistic regression

## 하이퍼파라미터 튜닝용 함수
def logistic_tuning(X_train, y_train, params):
    #model = LogisticRegression(random_state = 99)
    model = LogisticRegression()
    # 파라미터 튜닝(train data 전체를 넣어서 5-fold cv)
    grid = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid.best_estimator_

#%%time
logistic_param = {'C':[(4.0+x/10.0) for x in range(20)], 'max_iter': [150+m for m in range(0,110,10)]}
logistic_tuning_result = logistic_tuning(X_train, y_train, params = logistic_param) #최적의 하이퍼파라미터 확인

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

KeyboardInterrupt: ignored

In [ ]:
#AdaBoost Classifier

## 하이퍼파라미터 튜닝용 함수
def abc_tuning(train_sprs, y, params):
    model = AdaBoostClassifier()
    # 파라미터 튜닝(train data 전체를 넣어서 5-fold cv)
    grid = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5)
    grid.fit(train_sprs, y)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid.best_estimator_

#%%time
abc_param = {'n_estimators':[50+n for n in range(0,100,10)],'learning_rate':[1.0 + 0.005*i for i in range(1,11)]}
abc_tuning_result = abc_tuning(X_train, y_train, params = abc_param) #최적의 하이퍼파라미터 확인.

In [ ]:
#gradient boosting

## 하이퍼파라미터 튜닝용 함수
def gb_tuning(train_sprs, y, params):
    model = GradientBoostingClassifier()
    # 파라미터 튜닝(train data 전체를 넣어서 5-fold cv)
    grid = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5)
    grid.fit(train_sprs, y)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid.best_estimator_

#%%time
gb_param = {'n_estimators': [100+n for n in range(10,100,10)], 'max_depth': [3+m for m in range(1,6)]}
gb_tuning_result = gb_tuning(X_train, y_train, params = gb_param) #최적의 하이퍼파라미터 확인.

In [ ]:
#최적의 하이퍼파라미터로 학습한 후 결과

after_result = {'logistic':0, 'abc': 0, 'gb': 0}

#{'C': 5.9, 'max_iter': 250}
logistic_model = LogisticRegression(C=5.9, max_iter=250)
logistic_model.fit(X_train,y_train)
logistic_y_pred = logistic_model.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, logistic_y_pred))
after_result['logistic'] = metrics.accuracy_score(y_test, logistic_y_pred)

#{'learning_rate': 1.05, 'n_estimators': 100}
abc_model = AdaBoostClassifier(learning_rate=1.05, n_estimators=100)
abc_model.fit(X_train,y_train)
abc_y_pred = abc_model.predict(X_test)
# Model Accuracy
print("AdaBoost Classifier Accuracy:",metrics.accuracy_score(y_test, abc_y_pred))
after_result['abc'] = metrics.accuracy_score(y_test, abc_y_pred)

#{'max_depth': 5, 'n_estimators': 190}
gb_model = GradientBoostingClassifier(max_depth=5, n_estimators=190)
gb_model.fit(X_train, y_train, sample_weight=None, monitor=None)
gb_y_pred = gb_model.predict(X_test)
#Model Accuracy
print("GradientBoosting Classifier Accuracy:",metrics.accuracy_score(y_test, gb_y_pred))
after_result['gb'] = metrics.accuracy_score(y_test, gb_y_pred)

Logistic Regression Accuracy: 0.7280919420138435
AdaBoost Classifier Accuracy: 0.728744939271255
GradientBoosting Classifier Accuracy: 0.7403682904531801


In [ ]:
#plot of before_result
names = list(before_result.keys())
values=[]
for name in names:
  values.append(before_result[name])

plt.figure(figsize=(3, 3))

plt.scatter(names, values)
plt.title('default_result')
#plt.show()
print(before_result)

plt.savefig('/content/drive/MyDrive/학부연구생/패션/output_image/default_result.png')
plt.close()

{'logistic': 0.723520961211963, 'abc': 0.7284837403682904, 'gb': 0.7269165469505028}


In [ ]:
#plot of after_result
names = list(after_result.keys())
values=[]
for name in names:
  values.append(after_result[name])

plt.figure(figsize=(3, 3))

plt.scatter(names, values)
plt.title('tuned_result')
#plt.show()
print(after_result)
plt.savefig('/content/drive/MyDrive/학부연구생/패션/output_image/tuned_result.png')
plt.close()

{'logistic': 0.7280919420138435, 'abc': 0.728744939271255, 'gb': 0.7403682904531801}


In [ ]:
logistic_model.get_params()

{'C': 5.9,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 250,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
abc_model.get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1.05,
 'n_estimators': 100,
 'random_state': None}

In [ ]:
gb_model.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 190,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}